Missing Values, dropping columns, rows, filling the missing values

In [93]:
#!pip install pyspark


In [94]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("handling data").getOrCreate()

In [95]:
data = spark.read.csv("project_spark.csv", header =True, sep=";")
data.show()

+----+----+------+
|Name| Age|Salary|
+----+----+------+
|  AA|  20| 45000|
|  BB|  30| 60000|
|  CC|  40| 80000|
|  EE|null| 40000|
+----+----+------+



Dropping na

In [96]:
#Column Drop
data.drop("Salary").show()

+----+----+
|Name| Age|
+----+----+
|  AA|  20|
|  BB|  30|
|  CC|  40|
|  EE|null|
+----+----+



In [97]:
#Dropping na
data.dropna().show()
data.na.drop().show()

+----+---+------+
|Name|Age|Salary|
+----+---+------+
|  AA| 20| 45000|
|  BB| 30| 60000|
|  CC| 40| 80000|
+----+---+------+

+----+---+------+
|Name|Age|Salary|
+----+---+------+
|  AA| 20| 45000|
|  BB| 30| 60000|
|  CC| 40| 80000|
+----+---+------+



In [98]:
#drop if all columns without na are above or equal to a threshold
#if there are not enough non na columns in a row
data.na.drop(how = "any", thresh=3).show()

+----+---+------+
|Name|Age|Salary|
+----+---+------+
|  AA| 20| 45000|
|  BB| 30| 60000|
|  CC| 40| 80000|
+----+---+------+



In [99]:
#Check na only in a specific column
data.na.drop(how = "any", subset = ["Age"]).show()

+----+---+------+
|Name|Age|Salary|
+----+---+------+
|  AA| 20| 45000|
|  BB| 30| 60000|
|  CC| 40| 80000|
+----+---+------+



In [100]:
#drop only if all columns are null
data.na.drop(how = "all").show()

#data.na.drop("any").show()

+----+----+------+
|Name| Age|Salary|
+----+----+------+
|  AA|  20| 45000|
|  BB|  30| 60000|
|  CC|  40| 80000|
|  EE|null| 40000|
+----+----+------+




Filling na



In [101]:
data.na.fill("missing", ["Age"]).show()

+----+-------+------+
|Name|    Age|Salary|
+----+-------+------+
|  AA|     20| 45000|
|  BB|     30| 60000|
|  CC|     40| 80000|
|  EE|missing| 40000|
+----+-------+------+



In [102]:
print(data.columns)
print(list(f'Imputed {i}' for i in data.columns))
print(list(map(lambda i: f'Imputed {i}', data.columns)))

['Name', 'Age', 'Salary']
['Imputed Name', 'Imputed Age', 'Imputed Salary']
['Imputed Name', 'Imputed Age', 'Imputed Salary']


In [124]:
from pyspark.ml.feature import Imputer

new_data = data.drop("Name")
new_data = new_data.withColumn("Age", new_data["Age"].cast("int"))
new_data = new_data.withColumn("Salary", new_data["Salary"].cast("int"))

imputer = Imputer(
    inputCols = new_data.columns,
    outputCols = list(map(lambda i: f'Imputed {i}', new_data.columns))
).setStrategy("mean")

In [125]:
imputer.fit(new_data).transform(new_data).show()

+----+------+-----------+--------------+
| Age|Salary|Imputed Age|Imputed Salary|
+----+------+-----------+--------------+
|  20| 45000|         20|         45000|
|  30| 60000|         30|         60000|
|  40| 80000|         40|         80000|
|null| 40000|         30|         40000|
+----+------+-----------+--------------+

